In [22]:
import os
os.environ['COMFYUI_SERVER_ADDRESS'] = '127.0.0.1:6000'

from utils.actions.prompt_to_image import prompt_to_image
from utils.actions.prompt_image_to_image import prompt_image_to_image
from utils.actions.load_workflow import load_workflow
from api.api_helpers import clear
import sys
import time
import pandas as pd
import re
import requests
from PIL import Image
from io import BytesIO
from api.api_helpers import generate_image_by_prompt, load_cache_models
from api.websocket_api import get_history
from tqdm.auto import tqdm
import re
import numpy as np
import cv2
import os
import time
import cv2
import numpy as np
import pandas as pd
import shutil

#doesn't work on this:
#{'str': 'score_9, score_8_up, score_7_up, score_6_up, 1girl, beautiful woman. she is You have long and wavy brown hair. You\'re eyes are hazel. You\'re 5\'10" tall. You have a slim build,, small waist, and a plump, fit, and firm butt. You\'re skin is a lightly tanned colou({posing|standing|walking|kneeling|squatting|sitting}:1) in {__lovescape-location-modern-city__|her bedroom}. {|her gaze on viewer and} she is {smiling|sensual and passionate|romantic and lovely|bold and brave|boldly and defiantly|tempting and seductive|shy and blush|guilty and pity|flirty and frivolity}. anime, vivid, colorful, best quality, explicit. (cleavage:1.2)'}

import re

def clear_prompt(input_string):
    # Updated regex to extract the value of 'str' while handling special characters
    pattern = r"\{'str':\s*'([^']*)'\}|\{'str':\s*\"([^\"]*)\"\}"
    match = re.search(pattern, input_string)
    if match:
        # Use group(1) or group(2) depending on which part matches
        result = match.group(1) if match.group(1) else match.group(2)
        return result
    else:
        print("No match found")
        print(input_string)
    return None


workflow = load_workflow('./workflows/optional_workflows/cache_preload_api.json')
load_cache_models(workflow)
df = pd.read_csv('dataframes/inference_2024-12-06.csv')
df

127.0.0.1:6000
Progess:  1 / 8  Tasks done
Progess:  2 / 8  Tasks done
Progess:  3 / 8  Tasks done
Progess:  4 / 8  Tasks done
Progess:  5 / 8  Tasks done
Progess:  6 / 8  Tasks done
Progess:  7 / 8  Tasks done
Progess:  8 / 8  Tasks done
Progess:  9 / 8  Tasks done


,generation_time,result_image,ip_adapter_image,is_sfw,time_stamp,composition,checkpoint_name,style,project,gender,positive_prompt,negative_prompt,width,height,correlation_id,generation_type,webui_img_path,ip_adapter_img_path
0,5.663999,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,True,2024-12-03T19:54:56.172895+00:00,custom,dreamshaperXL_v21TurboDPMSDE,simple,NaN,NaN,beautiful adult woman. she is dressed. she is ...,"nude, naked, pussy, nipples, anus, skimpy, coc...",768,1024,7cace4ea0aee3f599d87986fcc5c31f7,sfw_custom_None_simple_dreamshaperXL_v21TurboD...,photos_loaded/inference_load_data_2024-12-06/i...,NaN
1,4.550477,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,True,2024-12-01T21:45:52.983649+00:00,full,clampdxl_v30,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up, 1...","score_4, score_5, score_3, score_2, score_1, s...",768,1152,eba527e4551e66ce86e051161527fef8,sfw_full_None_simple_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,NaN
2,3.854289,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,False,2024-11-28T21:34:55.871419+00:00,full,clampdxl_v30,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up, 1...","score_4, score_5, score_3, score_2, score_1, s...",768,1152,e3427d4a5850235a6b8a8c6e95655423,nsfw_full_None_simple_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,NaN
3,4.146353,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,False,2024-12-03T14:38:50.598629+00:00,custom,clampdxl_v30,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up, 1...","score_4, score_5, score_3, score_2, score_1, s...",768,1152,4901f4c190615fc2a2e5d3329f249205,nsfw_custom_None_simple_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,NaN
4,7.610383,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,False,2024-12-02T09:38:22.938966+00:00,full,natvisNaturalVision_v10,simple,NaN,NaN,beautiful adult woman. she is nude. she is Fre...,"cock, penis, central composition, worst qualit...",768,1152,fdcfbe4d41e9b50c835beeb2b8377287,nsfw_full_None_simple_natvisNaturalVision_v10,photos_loaded/inference_load_data_2024-12-06/i...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,8.407893,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,False,2024-12-02T09:39:49.779014+00:00,full,natvisNaturalVision_v10,simple,NaN,NaN,beautiful adult woman. she is nude. she is Fre...,"cock, penis, central composition, worst qualit...",768,1152,af73109433cd5338d7eef7e8f70a56ee,nsfw_full_None_simple_natvisNaturalVision_v10,photos_loaded/inference_load_data_2024-12-06/i...,NaN
393,10.164294,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/3/c/c/3cc...,True,2024-11-28T22:16:25.883328+00:00,custom,dreamshaperXL_v21TurboDPMSDE,simple,NaN,NaN,beautiful adult woman. she is dressed. she is ...,"nude, naked, pussy, nipples, anus, skimpy, coc...",768,1024,be002e253e083f0b28f2e7da18cfd7ba,sfw_custom_None_simple_dreamshaperXL_v21TurboD...,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
394,3.964605,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,True,2024-12-01T22:25:21.774066+00:00,full,clampdxl_v30,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up, 1...","score_4, score_5, score_3, score_2, score_1, s...",768,1152,97f247d7fd77f96661c91c6d1391c6c9,sfw_full_None_simple_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,NaN
395,4.305218,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,True,2024-11-28T02:59:36.598708+00:00,AVATAR,clampdxl_v30,ANIME,LOVESCAPE,MALE,"score_9, score_8_up, score_7_up, score_6_up, 1...",central composition. hands. topless. nude. pie...,768,1024,9eba8eb5e4210b3b798bec3889c58b09,sfw_AVATAR_MALE_ANIME_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,NaN


In [23]:
# !mv inference_2024-12-06.csv dataframes/

In [ ]:
# def find_available_lora_by_name(loras_dir: str, name: str) -> str:
#     for file in os.listdir(loras_dir):
#         if name in file:
#             return file
#     return None


# def extract_and_remove_loras(input_string):
#     # Regex pattern to match lora substrings
#     pattern = r"<lora:([a-zA-Z0-9\-_.]+):([\d.]+)>"
    
#     # Find all matches in the string
#     matches = re.findall(pattern, input_string)
    
#     # Extract lora names and weights into a list
#     loras = [{"name": match[0], "weight": float(match[1])} for match in matches]
    
#     # Remove all lora substrings from the input string
#     cleaned_string = re.sub(pattern, "", input_string)
    
#     # Return the extracted loras and the cleaned string
#     return loras, cleaned_string.strip()


# def add_loras_to_workflow(workflow, loras):
#     key = find_workflow_key(workflow, "Power Lora Loader (rgthree)")
#     lora_idxes = [int(key.split('_')[-1]) for key in workflow[key]["inputs"] if key.startswith("lora_")]
#     if lora_idxes:
#         last_lora_idx = max(lora_idxes)
#     else:
#         last_lora_idx = 0
    
#     for lora in loras:
#         lora_path = find_available_lora_by_name("/home/kazanplova/ComfyUI/models/loras", lora["name"])
#         if lora_path is None or "ip-adapter" in lora_path:
#             # print(f"Lora {lora['name']} not found!")
#             continue
#         last_lora_idx += 1
#         workflow[key]["inputs"][f"lora_{last_lora_idx}"] = {
#             "on": True,
#             "lora": lora_path,
#             "strength": lora["weight"]
#         }
#     return workflow, last_lora_idx

# def find_workflow_key(workflow, key):
#     for item in workflow:
#         workflow_data = workflow[item]
#         if workflow_data["class_type"] == key:
#             if workflow_data["class_type"] == "CLIPTextEncode" and workflow_data["inputs"]["text"] != "":
#                 continue
#             return item

# def text2img(positive_prompt, negative_prompt, loras, workflow, width, height, **kwargs):

#     workflow[find_workflow_key(workflow, "DPRandomGenerator")]["inputs"]["text"] = positive_prompt
#     workflow[find_workflow_key(workflow, "CLIPTextEncode")]["inputs"]["text"] = negative_prompt
        

#     workflow[find_workflow_key(workflow, "EmptyLatentImage")]["inputs"] = {
#         "width": width,
#         "height": height,
#         "batch_size": 1
#     }

#     workflow, last_lora_idx = add_loras_to_workflow(workflow, loras)

#     workflow[find_workflow_key(workflow, "Seed (rgthree)")]["inputs"]["seed"] = 42

#     begin = time.time()
#     images = generate_image_by_prompt(workflow, save_previews=True)
#     end = time.time()
#     return images, end - begin, workflow


In [25]:
df[df['checkpoint_name'] == 'realDream_sdxlPony11']

,generation_time,result_image,ip_adapter_image,is_sfw,time_stamp,composition,checkpoint_name,style,project,gender,positive_prompt,negative_prompt,width,height,correlation_id,generation_type,webui_img_path,ip_adapter_img_path
11,7.386354,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/6/8/4/684...,False,2024-12-01T01:58:59.471674+00:00,custom,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,bfc20984a082255f903a3e5edfea98c7,nsfw_custom_None_simple_realDream_sdxlPony11,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
14,4.816297,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/f/7/b/f7b...,False,2024-12-01T18:38:12.675689+00:00,full,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,65b33ada3262b3095f7c71f2dbd9385d,nsfw_full_None_simple_realDream_sdxlPony11,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
25,6.467960,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/1/f/d/1fd...,False,2024-12-03T02:34:56.470748+00:00,full,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,76b968898172428e629da831efca53d7,nsfw_full_None_simple_realDream_sdxlPony11,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
43,8.458700,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/c/9/a/c9a...,False,2024-12-01T07:14:45.672019+00:00,custom,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,2dd0160b0a54f9bc9db86f07c5b43f42,nsfw_custom_None_simple_realDream_sdxlPony11,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
62,7.024241,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/c/9/a/c9a...,False,2024-12-01T07:15:36.513947+00:00,custom,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,7a6f5a751b83b02b940d3492b60226d1,nsfw_custom_None_simple_realDream_sdxlPony11,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
67,5.054838,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/1/f/d/1fd...,True,2024-12-03T02:31:52.694359+00:00,full,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,1c308cbecb34d7209991efa64d4500ce,sfw_full_None_simple_realDream_sdxlPony11,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
72,124.556314,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/e/f/7/ef7...,False,2024-12-03T17:56:56.972787+00:00,custom,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,2fca2cfcca5b5e8e4f4c8457660ab494,nsfw_custom_None_simple_realDream_sdxlPony11,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
75,7.365207,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/8/b/e/8be...,False,2024-12-02T07:53:47.896922+00:00,full,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,720da3a84d47d812eddadbb238cceffc,nsfw_full_None_simple_realDream_sdxlPony11,photos_load

In [27]:
from copy import deepcopy


IP_ADAPTER_PATH = '/home/kazanplova/ComfyUI/input/ip_adapter_img.png'

checkpoint_mapper_dict = {
    'ip_adapter': {
        "natvisNaturalVision_v10": './workflows/ip_adapter_workflows/nsfw_realistic_ip_adapter_api_caching.json',
        "clampdxl_v30": './workflows/ip_adapter_workflows/anime_ip_adapter_api_caching.json',
        "dreamshaperXL_v21TurboDPMSDE": './workflows/ip_adapter_workflows/sfw_realistic_ip_adapter_api_caching.json',
        "realDream_sdxlPony11": './workflows/ip_adapter_workflows//ponya_ip_adapter_api_caching.json'

    },
    "simple": {
        "natvisNaturalVision_v10": './workflows/simple_workflows/realistic_nsfw_lora_api.json',
        "clampdxl_v30": './workflows/simple_workflows/anime_workflow_fix_cache.json',
        "dreamshaperXL_v21TurboDPMSDE": './workflows/simple_workflows/realistic_sfw_fix_cache.json',
        "realDream_sdxlPony11": './workflows/simple_workflows/pony_avatar_cache.json'
}
}


def webui_to_comfy(sdwebui_test):
    checkpoint_name = sdwebui_test['checkpoint_name']
    print(checkpoint_name, "ip_adapter" if not pd.isna(sdwebui_test['ip_adapter_img_path']) else "simple")
    if not pd.isna(sdwebui_test['ip_adapter_img_path']):
        ip_adapter_img = Image.open(sdwebui_test['ip_adapter_img_path'])
        ip_adapter_img.save(IP_ADAPTER_PATH)
        workflow_path = checkpoint_mapper_dict['ip_adapter'][checkpoint_name]
        base_workflow = deepcopy(load_workflow(workflow_path))
        base_workflow[find_workflow_key(base_workflow, "LoadImage")]["inputs"]["image"] = IP_ADAPTER_PATH
    else:
        workflow_path = checkpoint_mapper_dict['simple'][checkpoint_name]
        base_workflow = deepcopy(load_workflow(workflow_path))

    loras, cleaned_string = extract_and_remove_loras(sdwebui_test['positive_prompt'])
    # webui_conf = eval(sdwebui_test['webui_config'])
    # width, height = webui_conf['Size'].split('x')

    return {
        "positive_prompt": cleaned_string,
        "negative_prompt": sdwebui_test['negative_prompt'],
        "loras": loras,
        "workflow": base_workflow,
        "width": sdwebui_test['width'],
        "height": sdwebui_test['height'],
        "workflow_path": workflow_path
    }

def save_images(generated_img, sdwebui_img, ip_adapter_img, item_idx, inference_dir, webui_title, comfy_title):
    sdwebui_img = Image.open(sdwebui_img)
    ip_adapter_img = Image.open(ip_adapter_img) if not(pd.isna(ip_adapter_img)) else None
    
    sdwebui_img_np = np.array(sdwebui_img)
    generated_img_np = np.array(generated_img)  # Предполагается, что generate_image_by_prompt возвращает список изображений

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    # Calculate the average color of the background
    avg_color = np.mean(sdwebui_img_np, axis=(0, 1))

    # Determine if the background is light or dark
    is_light_bg = np.mean(avg_color) > 127

    # Set the text color to contrast with the background
    color = (0, 0, 0) if is_light_bg else (255, 255, 255)  # Black for light background, white for dark background
    thickness = 2

    cv2.putText(sdwebui_img_np, webui_title, (10, 30), font, font_scale, color, thickness, cv2.LINE_AA)

    cv2.putText(generated_img_np, comfy_title, (10, 30), font, font_scale, color, thickness, cv2.LINE_AA)

    if ip_adapter_img is not None:
        ip_adapter_img_np = np.array(ip_adapter_img)
        cv2.putText(ip_adapter_img_np, 'IP Adapter Image', (10, 30), font, font_scale, color, thickness, cv2.LINE_AA)
        ip_adapter_img_np = cv2.resize(ip_adapter_img_np, (sdwebui_img_np.shape[1], sdwebui_img_np.shape[0]))
        compared_images = cv2.hconcat([sdwebui_img_np, ip_adapter_img_np, generated_img_np])
    else:
        sdwebui_img_np = cv2.resize(sdwebui_img_np, generated_img_np.shape[:2][::-1])
        compared_images = cv2.hconcat([sdwebui_img_np, generated_img_np])
    compared_images = cv2.cvtColor(compared_images, cv2.COLOR_RGB2BGR)
    compared_path = os.path.join(inference_dir, f"compared_{item_idx}.png")
    cv2.imwrite(compared_path, compared_images)

    # Возвращаем пути к сохраненным изображениям, время выполнения и другую информацию
    return compared_path

In [28]:
cur_date = time.strftime("%Y-%m-%d")
inference_dir = f'inference_data/inference_{cur_date}'
shutil.rmtree(inference_dir, ignore_errors=True)
os.makedirs(inference_dir, exist_ok=True)
inference_dir

'inference_data/inference_2024-12-06'

In [29]:
df.generation_time.max()

132.1255241036415

In [30]:
df.dropna(subset='ip_adapter_image')

,generation_time,result_image,ip_adapter_image,is_sfw,time_stamp,composition,checkpoint_name,style,project,gender,positive_prompt,negative_prompt,width,height,correlation_id,generation_type,webui_img_path,ip_adapter_img_path
11,7.386354,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/6/8/4/684...,False,2024-12-01T01:58:59.471674+00:00,custom,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,bfc20984a082255f903a3e5edfea98c7,nsfw_custom_None_simple_realDream_sdxlPony11,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
12,11.927999,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/3/c/c/3cc...,True,2024-11-30T17:12:28.571517+00:00,custom,dreamshaperXL_v21TurboDPMSDE,simple,NaN,NaN,beautiful adult woman. she is dressed. she is ...,"nude, naked, pussy, nipples, anus, skimpy, coc...",768,1024,a1b2a5b297339fcfb8bf0877999a2407,sfw_custom_None_simple_dreamshaperXL_v21TurboD...,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
14,4.816297,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/f/7/b/f7b...,False,2024-12-01T18:38:12.675689+00:00,full,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,65b33ada3262b3095f7c71f2dbd9385d,nsfw_full_None_simple_realDream_sdxlPony11,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
16,3.494601,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/b/3/f/b3f...,False,2024-12-03T16:25:53.291408+00:00,custom,clampdxl_v30,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up, 1...","score_4, score_5, score_3, score_2, score_1, s...",768,1152,54eb5b701afc1a048d4c1601b862e2cf,nsfw_custom_None_simple_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
21,7.373967,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/2/9/9/299...,True,2024-12-01T22:56:19.278851+00:00,full,dreamshaperXL_v21TurboDPMSDE,simple,NaN,NaN,beautiful adult woman. she is dressed. she is ...,"nude, naked, pussy, nipples, anus, skimpy, coc...",768,1024,a18ce7b5517521ff52a2c7ba729b703b,sfw_full_None_simple_dreamshaperXL_v21TurboDPMSDE,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,4.007784,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/f/9/c/f9c...,True,2024-12-01T18:40:14.370146+00:00,custom,clampdxl_v30,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up, 1...","score_4, score_5, score_3, score_2, score_1, s...",768,1152,d6a59e5a692fc89c254a443db8ab267d,sfw_custom_None_simple_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
384,5.777644,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/1/3/9/139...,True,2024-11-28T10:10:58.872811+00:00,custom,realDream_sdxlPony11,simple,NaN,NaN,"score_9, score_8_up, score_7_up, score_6_up. (...","score_6, score_5, score_4, censored, central c...",768,1152,05378e5685443ba4865baf5385c2bb6d,sfw_custom_None_simple_realDream_sdxlPony11,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...
385,10.327565,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/0/b/b/0bb...,False,2024-12-02T04:43:18.474034+00:00,custom,natvisNaturalVision_v10,simple,NaN,NaN,beautiful adult woman. she is nude. she is __l...,"cock, penis, central composition, worst qualit...",768,1152,d2b56d7a7aec7

In [31]:
result_data = []
for i in tqdm(range(len(df))):
    sdwebui_test = df.iloc[i]
    
    comfy_request = webui_to_comfy(sdwebui_test)
    workflow = comfy_request['workflow']
    comfy_checkpoint_name = workflow[find_workflow_key(workflow, "CheckpointLoaderSimpleShared //Inspire")]["inputs"]["ckpt_name"]

    images, time_taken, workflow = text2img(**comfy_request)
    comfy_request['workflow'] = workflow
    webui_title = f"{'sfw' if sdwebui_test['is_sfw'] else 'nsfw'}_{sdwebui_test['checkpoint_name']}_{sdwebui_test['composition']}_webui"
    comfy_title = f"{comfy_checkpoint_name}_comfy"

    compared_images_path = save_images(images[0], sdwebui_test.webui_img_path,
                                        sdwebui_test.ip_adapter_img_path, i, inference_dir, webui_title, comfy_title)
    result_data.append(
        {
            "sdwebui_test": sdwebui_test,
            "comfy_request": comfy_request,
            "compared_images_path": compared_images_path,
            "comfy_time": time_taken
        }
    )


  0%|          | 0/397 [00:00<?, ?it/s]

dreamshaperXL_v21TurboDPMSDE simple
127.0.0.1:6000
Progess:  1 / 12  Tasks done
Progess:  2 / 12  Tasks done
Progess:  3 / 12  Tasks done
Progess:  4 / 12  Tasks done
Progess:  5 / 12  Tasks done
Progess:  6 / 12  Tasks done
Progess:  7 / 12  Tasks done
Progess:  8 / 12  Tasks done
Progess:  9 / 12  Tasks done
Progess:  10 / 12  Tasks done
In K-Sampler -> Step:  1  of:  6
In K-Sampler -> Step:  2  of:  6
In K-Sampler -> Step:  3  of:  6
In K-Sampler -> Step:  4  of:  6
In K-Sampler -> Step:  5  of:  6
In K-Sampler -> Step:  6  of:  6
Progess:  11 / 12  Tasks done
Progess:  12 / 12  Tasks done
Progess:  13 / 12  Tasks done
clampdxl_v30 simple
127.0.0.1:6000
Progess:  1 / 12  Tasks done
Progess:  2 / 12  Tasks done
Progess:  3 / 12  Tasks done
Progess:  4 / 12  Tasks done
Progess:  5 / 12  Tasks done
Progess:  6 / 12  Tasks done
Progess:  7 / 12  Tasks done
Progess:  8 / 12  Tasks done
Progess:  9 / 12  Tasks done
Progess:  10 / 12  Tasks done
In K-Sampler -> Step:  1  of:  8
In K-Sample

In [32]:
df.iloc[11].ip_adapter_img_path

'photos_loaded/inference_load_data_2024-12-06/item_11/ip_adapter_img.png'

In [33]:
!ls ./workflows/ip_adapter_workflows//ponya_ip_adapter_api_caching.json

./workflows/ip_adapter_workflows//ponya_ip_adapter_api_caching.json


In [34]:
!ls ./workflows/simple_workflows/

anime_workflow_fix_cache.json  realistic_nsfw_lora_api.json
pony_avatar_cache.json	       realistic_sfw_fix_cache.json


In [35]:
sdwebui_test

generation_time                                                 7.721484
result_image           https://s3.lovescape.com/mytwin-imagegenerator...
ip_adapter_image       https://cdn.lovescape.com/cdn/photos/1/6/b/16b...
is_sfw                                                              True
time_stamp                              2024-11-29T17:37:23.974186+00:00
composition                                                       custom
checkpoint_name                             dreamshaperXL_v21TurboDPMSDE
style                                                             simple
project                                                              NaN
gender                                                               NaN
positive_prompt        beautiful adult woman. she is dressed. she is ...
negative_prompt        nude, naked, pussy, nipples, anus, skimpy, coc...
width                                                                768
height                                             

In [36]:
df.checkpoint_name.value_counts()

checkpoint_name
clampdxl_v30                    182
dreamshaperXL_v21TurboDPMSDE    140
natvisNaturalVision_v10          40
realDream_sdxlPony11             35
Name: count, dtype: int64

In [37]:
comfy_request

{'positive_prompt': 'beautiful adult woman. she is dressed. she is caucasian, 19 yo, Brown hair, straight hairstyle, skinny build with (small breasts and small butt:.1). she is in bathroom happy, smiling sitting with spread legs high-waisted pencil skirt  wide view. highly detailed unretouched raw reddit mobile photograph with iso film grain. lingerie under clothes. (cleavage:1.2)',
 'negative_prompt': 'nude, naked, pussy, nipples, anus, skimpy, cock, penis, labia, transparent, butt, topless, ass, central composition, worst quality, low quality, low resolution, unrealistic, saturated, high contrast, blurred, absurd, surrealistic, cgi, render, painting, drawing, sketch, cartoon, anime, manga, 3d, watermark, signature, label, text, logo, worst body proportions, deformed, mutation, ugly, fused fingers, bad hands, missing fingers, a, an, the',
 'loras': [{'name': 'clothing_slider_v19_000000030', 'weight': 5.0},
  {'name': 'ip-adapter-faceid-plusv2_sdxl_lora', 'weight': 0.7}],
 'workflow': 

In [38]:
images

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=768x1024>]

In [39]:
final_df = pd.concat([df, pd.DataFrame(result_data)], axis=1)
final_df

,generation_time,result_image,ip_adapter_image,is_sfw,time_stamp,composition,checkpoint_name,style,project,gender,...,width,height,correlation_id,generation_type,webui_img_path,ip_adapter_img_path,sdwebui_test,comfy_request,compared_images_path,comfy_time
0,5.663999,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,True,2024-12-03T19:54:56.172895+00:00,custom,dreamshaperXL_v21TurboDPMSDE,simple,NaN,NaN,...,768,1024,7cace4ea0aee3f599d87986fcc5c31f7,sfw_custom_None_simple_dreamshaperXL_v21TurboD...,photos_loaded/inference_load_data_2024-12-06/i...,NaN,generation_time ...,{'positive_prompt': 'beautiful adult woman. sh...,inference_data/inference_2024-12-06/compared_0...,7.166830
1,4.550477,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,True,2024-12-01T21:45:52.983649+00:00,full,clampdxl_v30,simple,NaN,NaN,...,768,1152,eba527e4551e66ce86e051161527fef8,sfw_full_None_simple_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,NaN,generation_time ...,"{'positive_prompt': 'score_9, score_8_up, scor...",inference_data/inference_2024-12-06/compared_1...,3.232646
2,3.854289,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,False,2024-11-28T21:34:55.871419+00:00,full,clampdxl_v30,simple,NaN,NaN,...,768,1152,e3427d4a5850235a6b8a8c6e95655423,nsfw_full_None_simple_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,NaN,generation_time ...,"{'positive_prompt': 'score_9, score_8_up, scor...",inference_data/inference_2024-12-06/compared_2...,1.669710
3,4.146353,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,False,2024-12-03T14:38:50.598629+00:00,custom,clampdxl_v30,simple,NaN,NaN,...,768,1152,4901f4c190615fc2a2e5d3329f249205,nsfw_custom_None_simple_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,NaN,generation_time ...,"{'positive_prompt': 'score_9, score_8_up, scor...",inference_data/inference_2024-12-06/compared_3...,1.631791
4,7.610383,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,False,2024-12-02T09:38:22.938966+00:00,full,natvisNaturalVision_v10,simple,NaN,NaN,...,768,1152,fdcfbe4d41e9b50c835beeb2b8377287,nsfw_full_None_simple_natvisNaturalVision_v10,photos_loaded/inference_load_data_2024-12-06/i...,NaN,generation_time ...,{'positive_prompt': 'beautiful adult woman. sh...,inference_data/inference_2024-12-06/compared_4...,3.530532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,8.407893,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,False,2024-12-02T09:39:49.779014+00:00,full,natvisNaturalVision_v10,simple,NaN,NaN,...,768,1152,af73109433cd5338d7eef7e8f70a56ee,nsfw_full_None_simple_natvisNaturalVision_v10,photos_loaded/inference_load_data_2024-12-06/i...,NaN,generation_time ...,{'positive_prompt': 'beautiful adult woman. sh...,inference_data/inference_2024-12-06/compared_3...,3.477058
393,10.164294,https://s3.lovescape.com/mytwin-imagegenerator...,https://cdn.lovescape.com/cdn/photos/3/c/c/3cc...,True,2024-11-28T22:16:25.883328+00:00,custom,dreamshaperXL_v21TurboDPMSDE,simple,NaN,NaN,...,768,1024,be002e253e083f0b28f2e7da18cfd7ba,sfw_custom_None_simple_dreamshaperXL_v21TurboD...,photos_loaded/inference_load_data_2024-12-06/i...,photos_loaded/inference_load_data_2024-12-06/i...,generation_time ...,{'positive_prompt': 'beautiful adult woman. sh...,inference_data/inference_2024-12-06/compared_3...,10.176271
394,3.964605,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,True,2024-12-01T22:25:21.774066+00:00,full,clampdxl_v30,simple,NaN,NaN,...,768,1152,97f247d7fd77f96661c91c6d1391c6c9,sfw_full_None_simple_clampdxl_v30,photos_loaded/inference_load_data_2024-12-06/i...,NaN,generation_time ...,"{'positive_prompt': 'score_9, score_8_up, scor...",inference_data/inference_2024-12-06/compared_3...,3.438327
395,4.305218,https://s3.lovescape.com/mytwin-imagegenerator...,NaN,True,2024-11-28T02:59:36.598708+00:00,AVATAR,clampdxl_v30,ANIME,LOVESCAPE,MALE,...,768,1024,9eba8eb5e4210b3b798bec3889c58b09,sfw_AVATAR_MALE_ANIME_clampdxl_v30,photo

In [40]:
final_df.to_csv(f'dataframes/inference_{cur_date}.csv', index=False)

In [41]:
f'inference_{cur_date}.csv'

'inference_2024-12-06.csv'

In [19]:
# num = 22

# print(df.iloc[num].positive_prompt)
# print(df.iloc[num].negative_prompt)
# comfy_request = webui_to_comfy(df.iloc[num])
# print(comfy_request)
# print(df.iloc[num].generation_time)
# print(df)
# print(df.iloc[num].sdwebui_config)
# Image.open(df.iloc[num].webui_img_path)